In [12]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd

In [13]:
# Import data

# Path to course data folder
data_folder_fp = os.path.join('/',
                              'courses',
                              'EDS220', 
                              'data')

# Import countries shapefile
fp = os.path.join(data_folder_fp, 
                  'arctic_communities.geojson')
df = gpd.read_file(fp)

In [14]:
df.head(3)

,admin,country,n_communities,geometry
0,United States of America,US,115,"MULTIPOLYGON (((-132.74687 56.52568, -132.7576..."
1,United Kingdom,GB,96,"MULTIPOLYGON (((-2.66768 51.62300, -2.74214 51..."
2,Sweden,SE,133,"MULTIPOLYGON (((19.07646 57.83594, 18.99375 57..."


In [15]:
df.shape

(15, 4)

In [16]:
df.dtypes

admin              object
country            object
n_communities       int64
geometry         geometry
dtype: object

In [17]:
df.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [19]:
df.geom_type

0     MultiPolygon
1     MultiPolygon
2     MultiPolygon
3     MultiPolygon
4     MultiPolygon
5     MultiPolygon
6          Polygon
7          Polygon
8     MultiPolygon
9     MultiPolygon
10    MultiPolygon
11    MultiPolygon
12    MultiPolygon
13    MultiPolygon
14         Polygon
dtype: object

The output is a list of the geometry types for each row of data.

## 2. Check geometry types

In [20]:
# Check the geometry type of the df with an if-else statement
if df.geom_type.unique().tolist() == "Polygon":
    print("All features are polygons")
else:
    print(f"Multiple feature types: {df.geom_type.unique()}")

Multiple feature types: ['MultiPolygon' 'Polygon']


In [21]:
# Make a function that tests that all features are polygons
def check_polygons(data_frame):
    if df.geom_type.unique().tolist() == "Polygon":
        print("All features are polygons")
    else:
        print(f"Multiple feature types: {df.geom_type.unique()}")

In [ ]:
check_polygons(df)

## 3. Explode polygons

In [23]:
# transform the muiltipolygons into multiple observations with single-polygon
df = df.explode(column = 'geometry', index_parts = False)

The `explode()` method breaks up rows with milti-part geometry(polygons) into multiple rows with single_geometries

In [25]:
check_polygons(df)

Multiple feature types: ['Polygon']


In [26]:
df.head(3)

,admin,country,n_communities,geometry
0,United States of America,US,115,"POLYGON ((-132.74687 56.52568, -132.75762 56.5..."
0,United States of America,US,115,"POLYGON ((-132.77988 56.24727, -132.83096 56.2..."
0,United States of America,US,115,"POLYGON ((-134.31274 58.22891, -134.31987 58.2..."


In [ ]:
df.reset_index()

## 4. Compute minimum y-coordinate for polygons

In [35]:
# Select the first row of df using iloc. What kind of Python object is this?

print(df.iloc[[0]])
type(df.iloc[[0]])

                      admin country  n_communities  \
0  United States of America      US            115   

                                            geometry       miny  
0  POLYGON ((-132.74687 56.52568, -132.75762 56.5...  56.511035  


geopandas.geodataframe.GeoDataFrame

In [38]:
# Select the geometry of the first row of df. What kind of Python object is this?

print(df.iloc[[0]]['geometry'])
type(df.iloc[[0]]['geometry'])

0    POLYGON ((-132.74687 56.52568, -132.75762 56.5...
Name: geometry, dtype: geometry


geopandas.geoseries.GeoSeries

In [39]:
df.iloc[[0]]['geometry'].bounds['miny']

0    56.511035
Name: miny, dtype: float64

In [40]:
def min_y(geodf_row):
    return geodf_row.geometry.bounds[1]

In [41]:
df['miny'] = df.apply(min_y, axis = 1)

## 6. Filter, update CRS, and reproduce map

In [43]:
df[df['miny']] >= 40

# Update CRS
df = df[df['miny'] >= 40].to_crs('EPSG:3413')
df.plot()

KeyError: "None of [Index([56.511035156249996,       56.244140625,      58.2041015625,\n       60.312646484374994, 59.812646484374994,    60.034716796875,\n       60.358056640624994,      55.2587890625, 54.967333984374996,\n           55.05732421875,\n       ...\n          69.335693359375,      69.3884765625,      68.5849609375,\n          68.402197265625,    67.318212890625,     67.87880859375,\n               66.8578125,     67.98759765625,    69.539306640625,\n       51.265039062499994],\n      dtype='float64', length=481)] are in the [columns]"